# Phyling API

To use this notebook, you need to create a `.env` file with:

```env
PHYLING_API_KEY=your_api_key

# Server API:   https://api.app.phyling.fr
# Maxi-Hub API: 192.168.1.42:5001
PHYLING_API_URL="https://api.app.phyling.fr"
```

In [1]:
import json
import logging
import os
from dotenv import load_dotenv

from phyling.api import PhylingAPI

load_dotenv()
logging.getLogger().setLevel(logging.INFO)


# API connection

In [2]:
api = PhylingAPI(
    api_key=os.getenv("PHYLING_API_KEY"),
    url=os.getenv("PHYLING_API_URL", "https://api.app.phyling.fr/")
)

INFO:root:[API_CALL] 0.07s - GET http://localhost:5001/login - 200
INFO:root:PhylingAPI(baseurl=http://localhost:5001) -> Connected as tim.nicolas@phyling.fr


In [3]:
print(api.connected_user)

{'active': True, 'client_id': 2, 'created_date': '0000-00-00', 'externalAPI': {'nolio': {'connected': False}, 'sdh': {'connected': False}}, 'firstname': 'Tim', 'groups': [{'client_id': 2, 'id': 1, 'name': 'phyling'}], 'id': 10, 'lastname': 'Nicolas', 'mail': 'tim.nicolas@phyling.fr', 'psqs_id': '', 'roles': ['Admin', 'Athlete']}


# Examples

## Get the users

Several filters can be applied:
```python
def get_users(
    self,
    client_id: Union[int, None] = None,
    group_ids: list = [],
    role: Union[str, None] = None,
    active: bool = True,
    search: str = "",
    pageId: int = 1,
    pageSize: int = 0,
    soft: bool = False,
) -> Union[dict, None]:
    """Get the users from the API

    Args:
        client_id (int): clients id. None to select all
        group_ids (list): list of group ids. Empty to select all
        role (str, optional): The users roles (Admin, Coach or Athlete). None to select all
        active (bool, optional): The users active status. Defaults to True.
        search (str): The search string
        pageSize (int): Size of one page. Default: -1 (select all)
        pageId (int): Page id. Default: 1
        soft (bool): If true, only return minimal information on users

    Returns:
        dict: {
            "items": [User, ...],
            "total": int,
        }
    """
    pass
```

In [4]:
users = api.get_users(
    client_id=api.client_id,
)
print(users)

INFO:root:[API_CALL] 0.07s - POST http://localhost:5001/users/all - 200


{'items': [User(id=6, mail=jean-philippe.boucher@phyling.fr), User(id=5, mail=eric.busquet@phyling.fr), User(id=78, mail=romain.caux@phyling.fr), User(id=9, mail=thomas.chevallier@phyling.fr), User(id=692, mail=emmadebaud@gmail.com), User(id=3, mail=benoit.grillet@phyling.fr), User(id=2, mail=a.karamanoukian@phyling.fr), User(id=12, mail=alban.labbe@phyling.fr), User(id=8, mail=romain.labbe@phyling.fr), User(id=633, mail=tristan.lefebvre@phyling.fr), User(id=10, mail=tim.nicolas@phyling.fr), User(id=1, mail=admin@phyling.fr), User(id=11, mail=remy.roinson@phyling.fr), User(id=257, mail=romain.caux@gmail.com), User(id=7, mail=marie.segol@phyling.fr), User(id=663, mail=clement.thibierge@phyling.fr)], 'total': 16}


In [5]:
print(users["items"][0].to_dict())

{'active': True, 'client_id': 2, 'created_date': '0000-00-00', 'firstname': 'Jean-Philippe', 'groups': [{'client_id': 2, 'id': 1, 'name': 'phyling'}], 'id': 6, 'lastname': 'Boucher', 'mail': 'jean-philippe.boucher@phyling.fr', 'psqs_id': '', 'roles': ['Admin', 'Athlete']}


## Get records

Several filters can be applied:
```python
def get_records(
    self,
    type: str = "all",
    pageSize: int = 10,
    pageId: int = 1,
    onlyFavorite: bool = False,
    userIds: list = [],
    deviceIds: list = [],
    clientIds: list = [],
    groupIds: list = [],
    exerciseIds: list = [],
    sportIds: list = [],
    record_type: str = "",
    scenarioIds: list = [],
    minDate: str = "",
    maxDate: str = "",
) -> Union[dict, None]:
    """Get the records from the API

    Args:
        type (str): should be `all`, `new` or `associated`
        pageSize (int): Size of one page. Default: -1 (select all)
        pageId (int): Page id. Default: 1
        onlyFavorite (bool, optional): Select only favorites. Default to False.
        userIds (list): Users ids. Empty to select all
        deviceIds (list): devices ids. Empty to select all
        clientIds (list): clients ids. Empty to select all
        groupIds (list): groups ids. Empty to select all
        exerciseIds (list): exercises ids. Empty to select all
        sportIds (list): sport ids. Empty to select all
        record_type (str): record type. Can be seance, record, scenario, calib, miniphyling, fusion or video
        scenarioIds (list[int]): scenario ids. Empty to select all
        minDate (str): min record date. Empty to select all
        maxDate (str): max record date. Empty to select all

    Returns:
        dict: {
            "records": [Record, ...],
            "total": int,
        }
    """
    pass
```

In [6]:
recs = api.get_records(
    userIds=[api.connected_user["id"]],
)

INFO:root:[API_CALL] 0.10s - POST http://localhost:5001/records/all - 200


In [7]:
print(recs["records"][0])

Record(
    id=285,
    date=06/11/2025 11:56:35,
    record_type=scenario,
    decode_state=decoded,
    client_id=2,
    group_id=1,  # phyling
    user_ids=[10],  # Tim Nicolas
    sport_id=1,  # Défaut
    device_id=10300378,  # Maxi 378
    exercise_name=Scénario d'exemple,
    size=811 ko,
)


---
Download a record

```python
def download_record(
    self,
    rec_id: int,
    file_type: str,
    download_path: str,
    overwrite: bool = True,
    timeout: int = 180,
    **kwargs,
) -> bool:
    """
    Download the record.

    Args:
        rec_id (int): The ID of the record to download.
        file_type (str): The type of file to download. Can be one of the following:
            - "raw": raw data (.txt)
            - "decoded": decoded data (.csv)
            - "pdf": pdf report (.pdf)
            - "stats": stats report (.csv)
            - "specific_stats": specific stats report (.csv)
            - "video": video file (.mp4) -> only for video records
            - "zip": zip file (.zip)
        download_path (str): The path to save the downloaded file.
        overwrite (bool): If True, overwrite the file if it already exists. Default is False.
        timeout (int): The timeout for the request. Default is 180 seconds.
        **kwargs: Additional arguments for the request.
    """
    pass
```

In [8]:
api.download_record(
    rec_id=recs["records"][0]["id"],
    file_type="raw",
    download_path="/tmp/example_data.txt",
    overwrite=True,
)

INFO:root:[API_CALL] 0.05s - POST http://localhost:5001/records/285/file/raw - 200
INFO:root:File downloaded to: /tmp/example_data.txt


True

The decoding of a record in txt format is detailed in the notebook `decoder.ipynb`.